# Affective Positive  

In [ ]:
from decouple import config
import os
import pandas as pd

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = config('OPENAI_API_KEY'))
import json

In [ ]:
# Define the working directory
base_path = '/Users/jaredguerrero/Documents/Mis publicaciones/2025-1 Teoría GLMP/Code/AutomatedGLMP/src/affective'
os.chdir(base_path)

In [ ]:
with open(os.path.join(base_path, 'files', 'prompt_for_inference.txt'), 'r',  encoding='utf8') as f:
    prompt = f.read()
print(prompt)

In [ ]:
with open(os.path.join('files', 'positive_impression_base.json'), "r") as f:
    glmp_base = json.load(f)
glmp_base

In [ ]:
with open(os.path.join('files', 'measures.json'), "r") as f:
    measures = json.load(f)
glmp_base

In [ ]:
prompt += "\n\n Template: " + json.dumps(glmp_base)

In [ ]:
scores_df = pd.read_csv(os.path.join('input', 'turker_scores.csv'))
scores_df

In [ ]:
scores_df

In [ ]:
for i, row in scores_df.iterrows():
    #Convertimos cada fila en un diccionario, pero sin los campos 'Participant' y 'Overall'
    row_dict = row.to_dict()
    del row_dict['Participant']
    del row_dict['Overall']

    print(row_dict)

In [ ]:
conversation_history = [{"role": "assistant", "content": prompt}]

for i, row in scores_df.iterrows():
    #Verificamos si el participante ya tiene un archivo de salida
    if os.path.exists(os.path.join('output', f'output_{row["Participant"]}.txt')):
        continue
    else:
        print(f"Generating output for participant {row['Participant']}")
        #Convertimos cada fila en un diccionario, pero sin los campos 'Participant' y 'Overall'
        row_dict = row.to_dict()
        del row_dict['Participant']
        del row_dict['Overall']

        conversation_history.append({"role": "user", "content": json.dumps(row_dict)})
        response = client.chat.completions.create(
            model="o1-mini",
            messages=conversation_history,
        )

        assistant_response = response.choices[0].message.content
        conversation_history.append({"role": "assistant", "content": assistant_response})

        #Guardamos la salida en un archivo
        with open(os.path.join('output', f'output_{row["Participant"]}.txt'), 'w') as f:
            f.write(assistant_response)